In [1]:
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.nn.functional import sigmoid, relu

### Load Data

In [2]:
with open('1016data/sem_items.txt','r') as fid:
    names_items = np.array([l.strip() for l in fid.readlines()])
with open('1016data/sem_relations.txt','r') as fid:
    names_relations = np.array([l.strip() for l in fid.readlines()])
with open('1016data/sem_attributes.txt','r') as fid:
    names_attributes = np.array([l.strip() for l in fid.readlines()])
        
nobj = len(names_items)
nrel = len(names_relations)
nattributes = len(names_attributes)
print(f'num of objects: {nobj}')
print(f'num of relations: {nrel}')
print(f'num of attributes: {nattributes}')

num of objects: 75
num of relations: 14
num of attributes: 218


In [3]:
D = np.loadtxt('1016data/sem_data.txt')
input_pats = D[:,:nobj+nrel]
input_pats = torch.tensor(input_pats,dtype=torch.float)
output_pats = D[:,nobj+nrel:]
output_pats = torch.tensor(output_pats,dtype=torch.float)
N = input_pats.shape[0] # number of training patterns
print(f'num of training examples: {N}')

input_v = input_pats[0,:].numpy().astype('bool')
output_v = output_pats[0,:].numpy().astype('bool')
print('Example input pattern:')
print(input_v.astype('int'))
print('Example output pattern:')
print(output_v.astype('int'))
print("")
print("Which encodes...")
print('Item ',end='')
print(names_items[input_v[:75]])
print('Relation ',end='')
print(names_relations[input_v[75:]])
print('Attributes ',end='')
print(names_attributes[output_v])
print(f'num of attributes: {nattributes}')

num of training examples: 250
Example input pattern:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
Example output pattern:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Which encodes...
Item ['exercise']
Relation ['hasprerequisite']
Attributes ['danc' 'jog' 'lift weight' 'move' 'muscl' 'run']
num of attributes: 218


In [4]:
class Net(nn.Module):
    def __init__(self, rep_size, hidden_size, layer_1_size, hidden_rel_size):
        super(Net, self).__init__()
        self.itol1 = nn.Linear(nobj, layer_1_size)
        self.l1tor = nn.Linear(layer_1_size, rep_size)
        self.reltohr = nn.Linear(nrel, hidden_rel_size)
        self.cattoh = nn.Linear(rep_size+hidden_rel_size, hidden_size)
        self.htoa = nn.Linear(hidden_size, nattributes)

    def forward(self, x):
        x = x.view(-1,nobj+nrel)
        x_pat_item = x[:,:nobj]
        x_pat_rel = x[:,nobj:]
        layer1 = relu(self.itol1(x_pat_item))
        rep = relu(self.l1tor(layer1))
        rel = relu(self.reltohr(x_pat_rel))
        reprel = torch.cat((rep, rel), dim=1)
        hidden = relu(self.cattoh(reprel))
        output = self.htoa(hidden)
        return output, hidden, rep

In [5]:
mynet = Net(rep_size=75,hidden_size=200, layer_1_size=75, hidden_rel_size=30)
PATH = 'model_complex.pt'
mynet.load_state_dict(torch.load(PATH))
mynet.eval()

Net(
  (itol1): Linear(in_features=75, out_features=75, bias=True)
  (l1tor): Linear(in_features=75, out_features=75, bias=True)
  (reltohr): Linear(in_features=14, out_features=30, bias=True)
  (cattoh): Linear(in_features=105, out_features=200, bias=True)
  (htoa): Linear(in_features=200, out_features=218, bias=True)
)

In [6]:
df = pd.read_csv('bio_concepts.csv')
df.head(5)

,relation,head,tail
0,hasprerequisite,exercise,run
1,hassubevent,exercise,lose weight
2,isa,mammal,person
3,atlocation,food,restur
4,atlocation,food,cupboard


### Explore df

In [7]:
df[df['tail']=='organ']

,relation,head,tail
6,partof,liver,organ
31,isa,brain,organ
32,isa,eye,organ
45,partof,heart,organ
67,isa,animal,organ
68,isa,bacteria,organ
140,partof,tissue,organ
413,partof,organ,organ
425,partof,stomach,organ
431,isa,virus,organ


In [8]:
print('bladder could be used for generalizations, with known attributes like bladder isa organ and bladder hasa muscl')
filt = (df['head']=='bladder')
df[filt]

bladder could be used for generalizations, with known attributes like bladder isa organ and bladder hasa muscl


,relation,head,tail


### One-Hot Encoding

In [9]:
head_index = {}
for idx, i in enumerate(names_items):
    head_index[i]=idx
    
tail_index = {}
for idx, i in enumerate(names_attributes):
    tail_index[i]=idx
    
relation_index = {}
for idx, i in enumerate(names_relations):
    relation_index[i]=idx
    
assert len(head_index)==75
assert len(tail_index)==218
assert len(relation_index)==14

In [10]:
def one_hot_encoding(head, relation, tail, generalization=False):
    if generalization:
        vector = np.zeros(76+14)
        h_idx = 75  # the generalized concept will take the last spot
        r_idx = relation_index[relation]+76
    else:
        vector = np.zeros(75+14)
        h_idx = head_index[head]
        r_idx = relation_index[relation]+75
        
    output_vector = np.zeros(218)
    t_idx = tail_index[tail]  # used to identify the index of attributes
    output_vector[t_idx] = 1
    output_vector = torch.tensor(output_vector, dtype=torch.float).reshape(1,-1)
    
    vector[h_idx] = 1
    vector[r_idx] = 1
    vector = torch.tensor(vector,dtype=torch.float).reshape(1, -1)
    return vector, output_vector

### Generalization

In [11]:
class GeneralizationNet(nn.Module):
    def __init__(self, rep_size, hidden_size, layer_1_size, hidden_rel_size):
        super(GeneralizationNet, self).__init__()
        self.itol1 = nn.Linear(nobj+1, layer_1_size)
        self.l1tor = nn.Linear(layer_1_size, rep_size)
        self.reltohr = nn.Linear(nrel, hidden_rel_size)
        self.cattoh = nn.Linear(rep_size+hidden_rel_size, hidden_size)
        self.htoa = nn.Linear(hidden_size, nattributes)

    def forward(self, x):
        x = x.view(-1,nobj+1+nrel)
        x_pat_item = x[:,:nobj+1]
        x_pat_rel = x[:,nobj+1:]
        layer1 = relu(self.itol1(x_pat_item))
        rep = relu(self.l1tor(layer1))
        rel = relu(self.reltohr(x_pat_rel))
        reprel = torch.cat((rep, rel), dim=1)
        hidden = relu(self.cattoh(reprel))
        output = self.htoa(hidden)
        return output, hidden, rep

In [12]:
# load the state dict into newnet
newnet = GeneralizationNet(rep_size=75,hidden_size=200, layer_1_size=75, hidden_rel_size=30)
# copy the parameters from mynet to newnet
newnet.itol1.weight.data[:, :-1] = mynet.itol1.weight.data
newnet.itol1.bias.data = mynet.itol1.bias.data

newnet.l1tor.weight.data = mynet.l1tor.weight.data
newnet.l1tor.bias.data = mynet.l1tor.bias.data

newnet.reltohr.weight.data = mynet.reltohr.weight.data
newnet.reltohr.bias.data = mynet.reltohr.bias.data

newnet.cattoh.weight.data = mynet.cattoh.weight.data
newnet.cattoh.bias.data = mynet.cattoh.bias.data

newnet.htoa.weight.data = mynet.htoa.weight.data
newnet.htoa.bias.data = mynet.htoa.bias.data

In [13]:
for name, param in newnet.named_parameters():
    if name=='itol1.weight':
        param.requires_grad = True
    else:
        param.requires_grad = False

In [14]:
def train(net_, nepochs=5000):
    net_.train()
    n = training_input_pats.shape[0]
    for e in range(nepochs): # for each epoch
        error_epoch = 0.
        for p in range(n): # iterate through input patterns in random order
            net_.zero_grad() # reset gradient
            output, hidden, rep = net_(training_input_pats[p,:]) # forward pass
            target = training_output_pats[p,:]
            criterion = nn.BCEWithLogitsLoss(pos_weight=target)
            loss = criterion(output.reshape(218), target) # compute loss
            loss.backward() # compute gradient 
            optimizer.step() # update network parameters
            error_epoch += loss.item()
        error_epoch = error_epoch / float(n)        
        if e % 1000 == 0:
            print('epoch ' + str(e) + ' loss ' + str(round(error_epoch,7)))
        if e % 2000 == 0:
            print(f'grad of itol1 weight: {net_.itol1.weight.grad}')

In [15]:
train_input_pats_1, train_output_pats_1 = one_hot_encoding('bladder', 'isa', 'organ', generalization=True)
train_input_pats_2, train_output_pats_2 = one_hot_encoding('bladder', 'hasa', 'muscl', generalization=True)

training_input_pats = torch.cat((train_input_pats_1, train_input_pats_2), dim=0)
training_output_pats = torch.cat((train_output_pats_1, train_output_pats_2), dim=0)

### Training

In [16]:
learning_rate = 0.01
# criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(newnet.parameters(), lr=learning_rate, momentum=0.9) # stochastic gradient descent

train(newnet, 10000)

epoch 0 loss 0.1830097
grad of itol1 weight: tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0127],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0129],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0021],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0281],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0183],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0146]])
epoch 1000 loss 0.0003186
epoch 2000 loss 0.0002611
grad of itol1 weight: tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00, -2.4264e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00, -2.7908e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00, -2.6229e-05],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  1.5693e-04],
        [ 0.0000e+00, 

### Generalization

In [17]:
def generalize(head, network):
    network.eval()
    for rel in names_relations:
        input_vec, _ = one_hot_encoding(head, rel, tail='person', generalization=True)  # tail='person' is juat a place holder
        pred_idx = sigmoid(network(input_vec)[0]).detach().numpy()>=0.5
        pred = names_attributes[pred_idx.reshape(218)]
        print(f"{head} {rel}: {pred}")

In [18]:
generalize('bladder', newnet)

bladder atlocation: []
bladder capableof: []
bladder causes: ['pain']
bladder causesdesire: []
bladder hasa: ['muscl']
bladder hasprerequisite: ['move' 'muscl']
bladder hasproperty: []
bladder hassubevent: ['fall']
bladder isa: ['organ']
bladder madeof: ['muscl']
bladder motivatedbygoal: []
bladder partof: []
bladder receivesaction: []
bladder usedfor: []
